In [ ]:
import pandas as pd
from tqdm import tqdm
from math import log2

In [ ]:
df = pd.read_pickle('../data/reddit_corpus_balanced_filtered.gzip', compression='gzip')
df

In [ ]:
vocab = {}

for row in tqdm(df.iterrows(), total=len(df)):
    data = row[1]
    documents = data['documents']
    
    for doc_id, text, date, sub_reddit, labels in documents:
        if len(labels) == 1:
            if labels[0][1] == 0:
                    rn += 1
            else:
                fn += 1
            
            if ('brainwashed' in text):
                if labels[0][1] == 0:
                    not_rn_c += 1
                else:
                    not_fn_c += 1
            
            for word in text.split(' '):
                if word not in vocab:
                    vocab[word] = 0
                vocab[word] += 1

print(not_fn_c, not_rn_c, (not_fn_c/fn)/(not_rn_c/rn))

In [ ]:
c = 0
for word in dict(vocab):
    if vocab[word] < 100:
        del vocab[word]
        c += 1


vocab

In [ ]:
vocab_score = {}
rn = 0
fn = 0

for row in tqdm(df.iterrows(), total=len(df)):
    data = row[1]
    documents = data['documents']
    
    for doc_id, text, date, sub_reddit, labels in documents:
        if len(labels) == 1:
            if labels[0][1] == 0:
                    rn += 1
            else:
                fn += 1
            
            for word in vocab:
                if word in text:
                    if word not in vocab_score:
                        vocab_score[word] = {'rn': 0, 'fn': 0}
                    if labels[0][1] == 0:
                        vocab_score[word]['rn'] += 1
                    else:
                        vocab_score[word]['fn'] += 1
                        
for word in vocab_score:
    vocab_score[word]['score'] = (vocab_score[word]['fn']/fn)/(vocab_score[word]['fn']/fn + vocab_score[word]['rn']/rn)

vocab_score

In [ ]:
act = []
pred = []

for row in tqdm(df.iterrows(), total=len(df)):
    data = row[1]
    documents = data['documents']
    
    for doc_id, text, date, sub_reddit, labels in documents:
        if len(labels) == 1:
            if labels[0][1] == 0:
                act.append(0)
            else:
                act.append(1)
            
            rn = 0
            fn = 0
            done = False
            for word in text.split(' '):
                if word in vocab_score:
                    if vocab_score[word]['score'] == 0:
                        pred.append(0)
                        done = True
                        break
                    rn -= log2(1-vocab_score[word]['score'])
                    fn -= log2(vocab_score[word]['score'])
            
            if not done and rn <= fn:
                pred.append(0)
            else:
                pred.append(1)

In [ ]:
tp = sum(pred[i] == 1 and act[i] == 1 for i in range(len(act)))
tn = sum(pred[i] == 0 and act[i] == 0 for i in range(len(act)))
fp = sum(pred[i] == 1 and act[i] == 0 for i in range(len(act)))
fn = sum(pred[i] == 0 and act[i] == 1 for i in range(len(act)))

print("Pred\Act FN \t RN\nFN \t" + str(tp) + "\t" + str(fp) + "\nRN \t" + str(fn)+ "\t" + str(tn))

prec=tp/(tp+fp)
rec=tp/(tp+fn)
acc=(tp+tn)/(tp+tn+fp+fn)


print("\nprec:", prec)
print("rec:", rec)
print("acc:", acc)
print("F1:", 2*(prec*rec)/(prec+rec))